<a href="https://colab.research.google.com/github/diogofn1/Synthetic-Text-Data-Generator/blob/main/Synthetic_Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Important
This notebook was made to be run in google colab. This was done in order to allow users to make use of it regardless of having a good computer at home. However, with a small change in loading Hugging Face token, it can be run in a personal computer.
If run as a free Google Colab user, I recommend connection to a machine called 'GPUs: T4'. You can set that at the top right of your screen.

# Synthetic text data generator
The goal of this notebook is to build a synthetic text data generator using a open source model. Synthetic data is defined as artifiacially generated data that resembles real-world data. It is useful for multiple purposes, such as model training.
The model will leverage an open source model in order to keep it free of cost. The model will be "meta-llama/Meta-Llama-3.1-8B-Instruct", which is available at HuggingFace, is manageable by the power of Google Colab GPU T4, is able to receive instructions as a chatbot does, and yields good results.

In [ ]:
# Install requirements

!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate

In [ ]:
# Relevant imports

from google.colab import userdata, files
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
import gc
from IPython import display

In [ ]:
# Login to HuggingFace. HF_Token must be set in Google Colab Secrets
# If using on a personal machine, change this cell to load HF_TOKEN from a .env file
# One could also hardcode the token, but it is not recommended

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Quantization configuration
# Quantization allows less use of memory to load the model at the cost of some accuracy
# Without this, the notebook is likely to crash for lack of memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
# Load desired model

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
tokenizer.pad_token = tokenizer_llama.eos_token
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct",
                                             device_map="auto",
                                             quantization_config=quant_config)

In [ ]:
def generate_document(prompt, max_new_tokens=2000):

  """
  Generate documents based on the prompt given.
  Prompt (str): Instructions for the model.
  max_new_tokens (int): Maximum number of new tokens to generate.
  """

  prompt_updated = (
    "You are a synthetic data generator.\n\n"
    f"{prompt}\n"
    "Output your response in markdown format.\n"
  )
  inputs = tokenizer(prompt_updated, return_tensors="pt").to("cuda")
  input_length = inputs['input_ids'].shape[1]  # Number of tokens in the prompt
  outputs = model.generate(inputs['input_ids'], max_new_tokens=2000)
  generated_tokens = outputs[0][input_length:]
  generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

  return generated_text

## Sample uses

### 1. Generating a company summary

In [ ]:
prompt = (
    "Generate a document about Healthym, a company that sells healthy foods at local stores.\n"
    "This document should contain a couple of sections: summary about the company; a description of services it sells;\n"
    "a small background history; company values and mission.\n"
)

company_summary = generate_document(prompt, max_new_tokens=2000)

In [ ]:
display.Markdown(company_summary)

==============================================

Healthym: Nourishing Communities with Healthy Foods
=====================================================

### Summary

Healthym is a pioneering company dedicated to providing high-quality, healthy foods to local communities. We believe that everyone deserves access to nutritious food, regardless of their location or socioeconomic status. Our mission is to make healthy eating a reality for all, while promoting sustainability and supporting local economies.

### Services

Healthym offers a wide range of healthy food products, including:

* Fresh produce: seasonal fruits and vegetables sourced from local farmers
* Whole grains: artisanal bread, pasta, and rice from small-scale producers
* Plant-based protein: organic tofu, tempeh, and seitan from local suppliers
* Specialty foods: artisanal cheeses, fermented foods, and international cuisine
* Meal kits: pre-portioned ingredients and recipes for easy meal prep

Our products are carefully curated to meet the diverse needs of our customers, from busy professionals to families and individuals with dietary restrictions.

### Background History

Healthym was founded in 2015 by a group of passionate entrepreneurs who recognized the need for healthier food options in local communities. Our initial store opened in a small town in the Midwest, where we quickly gained a loyal customer base. As our popularity grew, we expanded to new locations, refining our business model and product offerings along the way.

Today, Healthym operates over 50 stores across the United States, with a strong presence in urban and rural areas. Our commitment to quality, sustainability, and community engagement has earned us a reputation as a trusted leader in the healthy food industry.

### Company Values and Mission

At Healthym, we live by the following values:

* **Quality**: We source only the best ingredients from local suppliers to ensure our products meet the highest standards.
* **Sustainability**: We strive to minimize our environmental footprint through eco-friendly packaging, reduced waste, and energy-efficient operations.
* **Community**: We believe in giving back to the communities we serve, through partnerships with local organizations and initiatives that promote food security and education.

Our mission is to:

* Provide access to healthy, nutritious food for all members of our communities
* Foster a culture of sustainability and environmental responsibility
* Support local economies and small-scale producers
* Educate and empower our customers to make informed food choices

At Healthym, we're dedicated to making a positive impact on the health and well-being of our customers, while promoting a more sustainable and equitable food system. Join us in our mission to nourish communities and create a healthier, happier world.

In [ ]:
# Save it locally

with open('company_summary.md', 'w') as f:
  f.write(company_summary)

files.download('company_summary.md')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 2. Generating multiple products

In [ ]:
products = ['seasonal fruits', "artisanal bread", "multivitamin", "organic meat"]

for product in products:

  prompt = (
      "Generate a document about products commercialized by Healthym, a company that sells healthy foods at local stores.\n"
      "This document should contain a couple of sections: product summary; a description of what it can be used for;\n"
      "product origin; if applicable, sample recipe in which it can be used, if applicable.\n"
      f"The product is: {product}.\n"
  )

  product_description = generate_document(prompt, max_new_tokens=2000)

  with open(f'{product}.md', 'w') as f:
    f.write(product_description)
  files.download(f'{product}.md')

In [ ]:
# Product file sample

with open('seasonal fruits.md', 'r') as f:
  file = f.read()

display.Markdown(file)

---

# Product Summary

**Seasonal Fruits**
=====================

Healthym's seasonal fruits are a collection of fresh and nutritious fruits available at local stores. Our selection includes a variety of fruits that are in season, ensuring maximum freshness and flavor.

# Product Description

**Description**
------------

Seasonal fruits can be used in a variety of ways, including:

*   Eating fresh as a snack or dessert
*   Adding to salads, yogurt, or oatmeal for extra flavor and nutrition
*   Using in smoothies or juices for a quick and healthy drink
*   Incorporating into baked goods, such as muffins or cakes

# Product Origin

**Origin**
----------

Our seasonal fruits are sourced from local farmers and suppliers who adhere to sustainable and environmentally friendly practices. We prioritize supporting local economies and promoting eco-friendly agriculture.

# Sample Recipe

**Seasonal Fruit Salad**
----------------------

Ingredients:

*   1 cup mixed seasonal fruits (such as strawberries, blueberries, grapes, and pineapple)
*   2 tablespoons honey
*   1 tablespoon lemon juice
*   1/4 cup chopped fresh mint

Instructions:

1.  In a large bowl, combine the mixed seasonal fruits.
2.  In a small bowl, whisk together the honey and lemon juice.
3.  Pour the honey-lemon mixture over the fruits and toss to coat.
4.  Sprinkle the chopped fresh mint over the top.
5.  Serve chilled or at room temperature.

This recipe showcases the versatility of seasonal fruits and highlights their use in a simple and delicious salad. Enjoy!

## Final remarks

This notebook can be use to generate synthetic text data for multiple purposes. The examples dispalyed here showcases its use to generate artificial data for a company called Healthym: one file for a company descrition and multiple files describing the product it offers. The user can adapt the parameters of the function generate_document() to fit their needs and address other contexts.